In [1]:
import time
import os
import numpy as np
import pandas as pd
from module import in_fun
from matplotlib import pyplot as plt
from module.torch import BinaryClassifier
from datetime import datetime

%matplotlib widget

/git/trajectory-correlation/module/in_fun.py:20: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if len(cur) is 0:
/git/trajectory-correlation/module/in_fun.py:30: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if len(cur) is 0:


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

In [3]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")

print(device)

cpu


In [4]:
net_diff = BinaryClassifier(input_size = 20)
net_each = BinaryClassifier(input_size = 40)
net_diff.load_state_dict(torch.load("./model/diff"))
net_each.load_state_dict(torch.load("./model/each"))

<All keys matched successfully>

In [5]:
data_path = "data/210423/"
file_list = os.listdir(data_path)
file_list_csv = [file for file in file_list if file.endswith(".csv")]
print(file_list_csv)

['00_confirmed.csv', '01_1250.csv', '02_1233.csv', '03_1210.csv', '04_1239.csv', '05_1241.csv', '06_1251_cs.csv', '07_1251_th.csv']


In [6]:
data_list = []
for file_csv in file_list_csv:
    data_list.append(pd.read_csv(
            data_path + file_csv,
            header=None,
        ).to_numpy())

In [7]:
plt.figure()
for idx, data in enumerate(data_list):
    plt.scatter(pd.to_datetime(data[:,0]/1000, unit='s'), np.ones_like(data[:,0])*(idx+1), label = file_list_csv[idx], s = 5)
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
plt.figure()
data=data_list[2]
plt.scatter(pd.to_datetime(data[:,0]/1000, unit='s'), data[:,42], s = 5)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# for idx, data in enumerate(data_list):
    print(f"{file_list_csv[idx][:-4]} start time : {datetime.fromtimestamp(data[0,0]/1000)}")

In [9]:
start_time = datetime(2021, 4, 23, hour=6, minute=20, second=0)
end_time = datetime(2021, 4, 23, hour=9, minute=20, second=0)
start_timestamp = time.mktime(start_time.timetuple())
end_timestamp = time.mktime(end_time.timetuple())
print(start_time)
print(start_timestamp)
print(end_time)
print(end_timestamp)
for idx in np.arange(len(data_list)):
    print(np.shape(data_list[idx]))
    data_list[idx] = data_list[idx][data_list[idx][:,0] >= start_timestamp * 1000]
    data_list[idx] = data_list[idx][data_list[idx][:,0] <= end_timestamp * 1000]
    
    print(np.shape(data_list[idx]))

2021-04-23 06:20:00
1619158800.0
2021-04-23 09:20:00
1619169600.0
(377, 101)
(360, 101)
(393, 101)
(360, 101)
(387, 101)
(360, 101)
(398, 101)
(360, 101)
(399, 101)
(360, 101)
(398, 101)
(360, 101)
(382, 101)
(360, 101)
(380, 101)
(360, 101)


In [10]:
plt.figure()
for idx, data in enumerate(data_list):
    plt.scatter(pd.to_datetime(data[:,0]/1000, unit='s'), np.ones_like(data[:,0])*(idx+1), label = file_list_csv[idx], s=5)
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
const_val = -200
start_time = time.time()
for idx in range(np.shape(data_list[0])[0]):
    confirmed_time = data_list[0][idx, 0]
    confirmed_ble = data_list[0][idx, 1:41] 
    confirmed_wifi = data_list[0][idx, 41:81]
    # confirmed part
    ## ble part
    if confirmed_ble[1] == 0:
        print("no confirmed ble")
        confirmed_ble_index = None
    else:
        confirmed_ble_index = True
        
    ## wifi part
    if confirmed_wifi[1] == 0:
        print("no confirmed wifi")
        confirmed_wifi_index = None
    else:
        confirmed_wifi_index = True
    # contact part
    
    ## ble part
    result_ble = []
    if confirmed_ble_index is not None:
        for contact in data_list[1:]:
            contact_ble = contact[idx, 1:41] 
            if contact_ble[1] == 0:
                result_ble.append("no contact ble")
            else:
                inp_diff, inp_each = in_fun.test_matcher(confirmed_ble, contact_ble)
#                 print(confirmed_ble)
#                 print(contact_ble)
#                 print(inp_each*100 -100)
                
                result_diff = net_diff(torch.from_numpy(inp_diff).float())
                result_each = net_each(torch.from_numpy(inp_each).float())
#                 result = np.array([result_diff, result_each]) >= 0.5
                result = result_each >= 0.4
                result_ble.append(result)
                
    ## wifi part
    result_wifi = []
    if confirmed_wifi_index is not None:
        for contact in data_list[1:]:
            contact_wifi = contact[idx, 41:81]
            if contact_wifi[1] == 0:
                result_wifi.append("no contact wifi")
            else:
                inp_diff, inp_each = in_fun.test_matcher(confirmed_wifi, contact_wifi)
                
                result_diff = net_diff(torch.from_numpy(inp_diff).float())
                result_each = net_each(torch.from_numpy(inp_each).float())
#                 result = np.array([result_diff, result_each]) >= 0.5
                result = result_each >= 0.5
                result_wifi.append(result)
                    
        
    print(f"시각 : {pd.to_datetime(confirmed_time/1000, unit='s')}")
    if len(result_ble) > 0:
#         print(f"ble 결과 : {result_ble}")
        for idx, result in enumerate(result_ble):
            if result == True:
                print(f"{file_list_csv[idx+1]} contacted")
#     if len(result_wifi) > 0:
#         print(f"wifi 결과 : {result_wifi}")
    
print(f"총 소요시간 : {time.time()-start_time}초")

시각 : 2021-04-23 06:20:00.002000128
07_1251_th.csv contacted
시각 : 2021-04-23 06:20:30.002000128
01_1250.csv contacted
07_1251_th.csv contacted
시각 : 2021-04-23 06:21:00.002000128
01_1250.csv contacted
07_1251_th.csv contacted
시각 : 2021-04-23 06:21:30.002000128
01_1250.csv contacted
07_1251_th.csv contacted
시각 : 2021-04-23 06:22:00.002000128
01_1250.csv contacted
07_1251_th.csv contacted
시각 : 2021-04-23 06:22:30.000999936
01_1250.csv contacted
07_1251_th.csv contacted
시각 : 2021-04-23 06:23:00.000999936
01_1250.csv contacted
07_1251_th.csv contacted
시각 : 2021-04-23 06:23:30.000999936
07_1251_th.csv contacted
시각 : 2021-04-23 06:24:00.000999936
07_1251_th.csv contacted
시각 : 2021-04-23 06:24:30.000999936
01_1250.csv contacted
07_1251_th.csv contacted
시각 : 2021-04-23 06:25:00.002000128
01_1250.csv contacted
07_1251_th.csv contacted
시각 : 2021-04-23 06:25:30.002000128
07_1251_th.csv contacted
시각 : 2021-04-23 06:26:00.002000128
01_1250.csv contacted
07_1251_th.csv contacted
시각 : 2021-04-23 06:26: